In [20]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.impute import KNNImputer

import pandas as pd
from sklearn.impute import SimpleImputer

# Load the dataset for which you want to estimate missing values
data = pd.read_csv('TrainData1.csv', delimiter='\t')

# Replace "1.00000000000000e+99" with NaN for missing values
data = data.replace(1.00000000000000e+99, float('nan'))

# Handle missing values using the 'mean' strategy
imputer = SimpleImputer(strategy='mean')

# Perform imputation
imputed_data = imputer.fit_transform(data)

# Convert back to a DataFrame if needed
imputed_dataframe = pd.DataFrame(imputed_data, columns=data.columns)

# Save the imputed DataFrame to a new CSV file
imputed_dataframe.to_csv('Imputed_TrainData1.csv', index=False)

from sklearn.multioutput import MultiOutputClassifier

# Load data
multilabel_train_data = pd.read_csv('Imputed_TrainData1.csv')
multilabel_train_labels = pd.read_csv('TrainLabel1.csv')
multilabel_test_data = pd.read_csv('TestData1.csv')

# Model selection and training
clf = make_pipeline(StandardScaler(), MultiOutputClassifier(RandomForestClassifier()))

clf.fit(imputer.fit_transform(multilabel_train_data), multilabel_train_labels)

# Prediction
predictions = clf.predict(imputer.transform(multilabel_test_data))

# Save predictions
pd.DataFrame(predictions).to_csv('SarahOBrienMultilabelClassification.txt', index=False, header=False)


# Load data with tab ('\t') delimiter
"""
train_data = pd.read_csv('TrainData1.csv', delimiter='\t')
train_labels = pd.read_csv('TrainLabel1.csv')
test_data = pd.read_csv('TestData1.csv', delimiter='\t')

# Replace "1.00000000000000e+99" with NaN for missing values
test_data = test_data.replace(1.00000000000000e+99, float('nan'))

# Ensure that columns in test data match those used during training
matching_columns = train_data.columns.intersection(test_data.columns)
test_data = test_data[matching_columns]

# Handle missing values using the 'mean' strategy
imputer = SimpleImputer(strategy='mean')

# Model selection and training
clf = make_pipeline(StandardScaler(), RandomForestClassifier())

# Combining preprocessing and model training
clf.fit(imputer.fit_transform(train_data), train_labels.values.ravel())

# Prediction
predictions = clf.predict(imputer.transform(test_data))

# Save predictions
pd.DataFrame(predictions).to_csv('SarahOBrienClassification1.txt', index=False, header=False)

"""


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- 3.84110595372964	1.73247417728119	1.71180722904119	2.41510695648807	3.00856184996988	3.06857172981411	3.33656779368507	3.47587884472798	3.59165551348753	3.70552418016734	1.68259629146055	1.91624333046534	2.37206467071813	2.37516960885164	2.48314493287213	1.27669152884504	3.26617523003128	1	1	3.48984462031632	2.23197902683150	1	1.65137494391304	2.45689718744935	2.77001205462655	2.86800928127859	2.29382643838618	2.16746502884309	3.54993587041630	1	2.67384997734295	2.00021709297223	3.02105278506516	2.38738982633873	1.89359533381988	1.05652372407910	1	1.97400479689741	2.33805787541976	3.29395226421916	3.00856184996988	3.21125940781845	2.61550817152816	3.39976195561922	1	1	1.43552585149865	3.09060365753608	2.04512730656803	3.36765404664741	1.69749088717106	2.14715052521007	2.80106052984786	3.34498533884836	3.31066782732285	3.05938542292466	3.65985727647127	2.06751720190368	3.310